In [15]:
%%time
import cx_Oracle
import numpy as np
import pandas as pd

#고객별 2014년 2015년 총구매액 및 평균구매액 출력
conn=cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
cursor = conn.cursor()

Wall time: 135 ms


In [16]:
df_one = pd.read_csv('기존고객등급횟수(진짜).csv',encoding='cp949')

In [17]:
df_one = df_one[['고객번호']]

In [18]:
# 기존고객
df_one.head()

,고객번호
0,1
1,2
2,3
3,4
4,6


In [70]:
# 편의품 1-6분기 객단가
cursor.execute('select * from code_11')
x = cursor.fetchall()

df = pd.DataFrame(x, columns=['고객번호','편의품객단가'])
df

,고객번호,편의품객단가
0,00001,21901
1,00002,27962
2,00003,2560
3,00004,7911
4,00005,12288
...,...,...
19356,19379,7270
19357,19380,10892
19358,19381,6641
19359,19382,4301


In [71]:
# 선매품 1-6분기 객단가
cursor.execute('select * from code_22')
x = cursor.fetchall()

df1 = pd.DataFrame(x, columns=['고객번호','선매품객단가'])
df1

,고객번호,선매품객단가
0,00001,217998
1,00002,255461
2,00003,56400
3,00004,78142
4,00005,103684
...,...,...
19276,19378,75352
19277,19379,81079
19278,19380,19361
19279,19381,32747


In [72]:
# 전문품 1-6분기 객단가
cursor.execute('select * from code_33')
x = cursor.fetchall()

df2 = pd.DataFrame(x, columns=['고객번호','전문품객단가'])
df2

,고객번호,전문품객단가
0,00001,509218
1,00002,389293
2,00005,58715
3,00006,55947
4,00007,1268483
...,...,...
8745,19362,91685
8746,19364,1199002
8747,19372,194901
8748,19375,31201


In [73]:
# 기타 1-6분기 객단가
cursor.execute('select * from code_44')
x = cursor.fetchall()

df3 = pd.DataFrame(x, columns=['고객번호','기타객단가'])
df3

,고객번호,기타객단가
0,00001,19632
1,00002,58860
2,00004,13821
3,00005,18816
4,00006,12790
...,...,...
17995,19377,21166
17996,19378,16211
17997,19379,10146
17998,19381,4409


In [74]:
df = df.astype('int64')

In [75]:
df1 = df1.astype('int64')

In [76]:
df2 = df2.astype('int64')

In [77]:
df3 = df3.astype('int64')

In [78]:
df_two = pd.merge(df_one,df,on='고객번호',how ='left')

In [79]:
df_three = pd.merge(df_two,df1,on='고객번호',how ='left')

In [80]:
df_four = pd.merge(df_three,df2,on='고객번호',how ='left')

In [81]:
df_five = pd.merge(df_four,df3,on='고객번호',how ='left')

In [82]:
df_five

,고객번호,편의품객단가,선매품객단가,전문품객단가,기타객단가
0,1,21901,217998.0,509218.0,19632.0
1,2,27962,255461.0,389293.0,58860.0
2,3,2560,56400.0,NaN,NaN
3,4,7911,78142.0,NaN,13821.0
4,6,7476,76643.0,55947.0,12790.0
...,...,...,...,...,...
18873,19333,5129,17502.0,NaN,9025.0
18874,19334,5869,17851.0,NaN,8129.0
18875,19335,7472,24181.0,NaN,15510.0
18876,19336,34778,105801.0,195934.0,15550.0


In [83]:
df_five = df_five.fillna(0)

In [84]:
df_five

,고객번호,편의품객단가,선매품객단가,전문품객단가,기타객단가
0,1,21901,217998.0,509218.0,19632.0
1,2,27962,255461.0,389293.0,58860.0
2,3,2560,56400.0,0.0,0.0
3,4,7911,78142.0,0.0,13821.0
4,6,7476,76643.0,55947.0,12790.0
...,...,...,...,...,...
18873,19333,5129,17502.0,0.0,9025.0
18874,19334,5869,17851.0,0.0,8129.0
18875,19335,7472,24181.0,0.0,15510.0
18876,19336,34778,105801.0,195934.0,15550.0


In [85]:
df_five =df_five.set_index('고객번호')

In [117]:
df_five.rename(columns={'편의품객단가':'14_1','선매품객단가':'14_2','전문품객단가':'14_3','기타객단가':'14_4'},inplace=True)
df_five.head()

,14_1,14_2,14_3,14_4
고객번호,,,,
1,21901,217998.0,509218.0,19632.0
2,27962,255461.0,389293.0,58860.0
3,2560,56400.0,0.0,0.0
4,7911,78142.0,0.0,13821.0
6,7476,76643.0,55947.0,12790.0


In [119]:
df_five2 = df_five.copy()

In [120]:
aa=df_five2['14_1'].sort_values(ascending=False)
cuts = pd.qcut(aa,10,labels=np.arange(1,11)[::-1])
cc=pd.concat([aa,cuts],axis=1)
cc

,14_1,14_1
고객번호,,
7957,3096249,1
10487,2125641,1
11763,1466619,1
6207,1423833,1
3833,1348691,1
...,...,...
18510,1570,10
18665,1516,10
14199,1418,10


In [121]:
aa=df_five2['14_2'].sort_values(ascending=False)
cuts = pd.qcut(aa,10,labels=np.arange(1,11)[::-1])
cc=pd.concat([aa,cuts],axis=1)
cc

,14_2,14_2
고객번호,,
15999,5437671.0,1
14599,5436135.0,1
16406,3362859.0,1
8079,3360443.0,1
8994,2719242.0,1
...,...,...
19171,0.0,10
12163,0.0,10
13182,0.0,10


In [122]:
aa=df_five2['14_4'].sort_values(ascending=False)
cuts = pd.qcut(aa,10,labels=np.arange(1,11)[::-1])
cc=pd.concat([aa,cuts],axis=1)
cc

,14_4,14_4
고객번호,,
16406,2262992.0,1
15999,2003893.0,1
16014,1637512.0,1
11999,760035.0,1
1933,659434.0,1
...,...,...
14548,0.0,10
14545,0.0,10
18279,0.0,10


# 비율 구하기

In [86]:
df_five

,편의품객단가,선매품객단가,전문품객단가,기타객단가
고객번호,,,,
1,21901,217998.0,509218.0,19632.0
2,27962,255461.0,389293.0,58860.0
3,2560,56400.0,0.0,0.0
4,7911,78142.0,0.0,13821.0
6,7476,76643.0,55947.0,12790.0
...,...,...,...,...
19333,5129,17502.0,0.0,9025.0
19334,5869,17851.0,0.0,8129.0
19335,7472,24181.0,0.0,15510.0


In [88]:
df_five2 = df_five.copy()

In [91]:
df_five2

,편의품객단가,선매품객단가,전문품객단가,기타객단가
고객번호,,,,
1,21901,217998.0,509218.0,19632.0
2,27962,255461.0,389293.0,58860.0
3,2560,56400.0,0.0,0.0
4,7911,78142.0,0.0,13821.0
6,7476,76643.0,55947.0,12790.0
...,...,...,...,...
19333,5129,17502.0,0.0,9025.0
19334,5869,17851.0,0.0,8129.0
19335,7472,24181.0,0.0,15510.0


In [89]:
df_five2 = df_five2.set_index('고객번호')

KeyError: "None of ['고객번호'] are in the columns"

In [92]:
df_five2 = df_five2.fillna(0)

In [93]:
df_five2

,편의품객단가,선매품객단가,전문품객단가,기타객단가
고객번호,,,,
1,21901,217998.0,509218.0,19632.0
2,27962,255461.0,389293.0,58860.0
3,2560,56400.0,0.0,0.0
4,7911,78142.0,0.0,13821.0
6,7476,76643.0,55947.0,12790.0
...,...,...,...,...
19333,5129,17502.0,0.0,9025.0
19334,5869,17851.0,0.0,8129.0
19335,7472,24181.0,0.0,15510.0


In [94]:
df_five2 = df_five2.div(df_five2.sum(axis=1),axis=0)

In [95]:
df_five2 = df_five2.fillna(0)

In [97]:
df_five2 = df_five2*100

In [102]:
df_five2 = df_five2.round(2)

In [100]:
df_five2 = df_five2/100

In [105]:
df_five2

,편의품객단가,선매품객단가,전문품객단가,기타객단가
고객번호,,,,
1,2.85,28.36,66.24,2.55
2,3.82,34.92,53.21,8.05
3,4.34,95.66,0.00,0.00
4,7.92,78.24,0.00,13.84
6,4.89,50.14,36.60,8.37
...,...,...,...,...
19333,16.20,55.29,0.00,28.51
19334,18.43,56.05,0.00,25.52
19335,15.84,51.27,0.00,32.89


In [106]:
df_five3 = df_five2.reset_index()

In [107]:
df_five3

,고객번호,편의품객단가,선매품객단가,전문품객단가,기타객단가
0,1,2.85,28.36,66.24,2.55
1,2,3.82,34.92,53.21,8.05
2,3,4.34,95.66,0.00,0.00
3,4,7.92,78.24,0.00,13.84
4,6,4.89,50.14,36.60,8.37
...,...,...,...,...,...
18873,19333,16.20,55.29,0.00,28.51
18874,19334,18.43,56.05,0.00,25.52
18875,19335,15.84,51.27,0.00,32.89
18876,19336,9.88,30.05,55.65,4.42


In [108]:
df_five3.columns = ['고객번호','편의품(%)','선매품(%)','전문품(%)','기타(%)']

In [109]:
df_five3

,고객번호,편의품(%),선매품(%),전문품(%),기타(%)
0,1,2.85,28.36,66.24,2.55
1,2,3.82,34.92,53.21,8.05
2,3,4.34,95.66,0.00,0.00
3,4,7.92,78.24,0.00,13.84
4,6,4.89,50.14,36.60,8.37
...,...,...,...,...,...
18873,19333,16.20,55.29,0.00,28.51
18874,19334,18.43,56.05,0.00,25.52
18875,19335,15.84,51.27,0.00,32.89
18876,19336,9.88,30.05,55.65,4.42


In [110]:
import csv
df_five3.to_csv('품목비율.csv',mode="w",encoding='euc-kr',index=False)

In [111]:
a = pd.read_csv('제휴사비율.csv',encoding='cp949')

In [112]:
a

,고객번호,A제휴사(%),B제휴사(%),C제휴사(%),D제휴사(%)
0,1,83.12,9.31,7.57,0.00
1,2,78.09,11.23,0.00,10.68
2,3,92.67,4.42,2.91,0.00
3,4,62.36,24.07,13.57,0.00
4,6,68.42,10.81,20.77,0.00
...,...,...,...,...,...
18873,19333,27.06,15.21,6.99,50.75
18874,19334,0.00,71.19,28.81,0.00
18875,19335,90.31,6.76,2.93,0.00
18876,19336,86.71,0.00,0.00,13.29


In [113]:
t = pd.merge(a,df_five3,on='고객번호')

In [114]:
import csv
t.to_csv('제휴사품목비율.csv',mode="w",encoding='euc-kr',index=False)